In [2]:
## install fsearch package 
!python -m pip install -q -e .

In [1]:

from fsearch.config import Config

config = Config(**{'Server': 'abc'})

In [2]:
from fsearch.utils import read_file

filepath = "samples/200k.txt"
file_contents = read_file(filepath)

In [2]:
from fsearch.algorithms import native_search, regex_search, rabin_karp_search, kmp_search, aho_corasick_search

query = "13;0;1;26;0;9;4;0;"
query = "13;0;1;"
## native_search
found = native_search(file_contents, query)
print('native_search:', found)
## regex_search
found = regex_search(file_contents, query)
print('regex_search:', found)
found = rabin_karp_search(file_contents, query)
print('rabin_karp:', found)
found = kmp_search(file_contents, query)
print('kmp_search:', found)
found = aho_corasick_search(file_contents, query)
print('aho_corasick:', found)

native_search: False
regex_search: False
rabin_karp: False
kmp_search: False
aho_corasick: False


In [16]:
# Example usage
contents = """13;0;1;26;0;9;4;0;
11;0;23;16;0;19;5;0;
9;0;1;6;0;10;5;0;
11;0;23;11;0;19;5;0;
17;0;1;26;0;7;3;0;
3;0;1;28;0;7;3;0;
4;0;1;28;0;8;3;0;
5;0;1;26;0;8;3;0; 
"""

query_1 = "11;0;23;11;0;19;5;0;"
query_2 = "11;0;23;11;0;19;5"

In [17]:
def rabin_karp_search(pattern, text):
    """
    Rabin-Karp algorithm to find a full line match of a pattern in the text.
    
    :param pattern: The pattern string to search for.
    :param text: The text in which to search for the pattern.
    :return: True if the pattern matches any full line in the text, otherwise False.
    """
    if not pattern or not text:
        return False

    # Define a prime number for the hash function
    prime = 101

    # Calculate the length of the pattern
    m = len(pattern)

    # Initialize hash values for pattern
    pattern_hash = 0

    # Calculate the hash value of the pattern
    for i in range(m):
        pattern_hash = (prime * pattern_hash + ord(pattern[i]))

    lines = text.split('\n')
    for line in lines:
        n = len(line)
        if n != m:
            continue
        
        # Initialize hash value for current line
        current_hash = 0
        for i in range(m):
            current_hash = (prime * current_hash + ord(line[i]))

        # Compare the hash values
        if pattern_hash == current_hash:
            # Check for exact match to avoid hash collision
            if line == pattern:
                return True

    return False


print('rabin_karp (q1):', rabin_karp_search(query_1, contents))  # Output: True
print('rabin_karp (q2):', rabin_karp_search(query_2, contents))  # Output: False


rabin_karp (q1): True
rabin_karp (q2): False


In [18]:
def kmp_search(pattern, text):
    """
    KMP algorithm to find a full line match of a pattern in the text.

    :param pattern: The pattern string to search for.
    :param text: The text in which to search for the pattern.
    :return: True if the pattern matches any full line in the text, otherwise False.
    """
    def compute_lps(pattern):
        """
        Compute the longest prefix which is also suffix array (lps) for the pattern.

        :param pattern: The pattern string.
        :return: The lps array.
        """
        m = len(pattern)
        lps = [0] * m
        length = 0
        i = 1

        while i < m:
            if pattern[i] == pattern[length]:
                length += 1
                lps[i] = length
                i += 1
            else:
                if length != 0:
                    length = lps[length - 1]
                else:
                    lps[i] = 0
                    i += 1

        return lps

    def kmp_search_line(pattern, line):
        """
        KMP search for a pattern in a single line.

        :param pattern: The pattern string.
        :param line: The line of text.
        :return: True if the pattern matches the full line, otherwise False.
        """
        m = len(pattern)
        n = len(line)
        
        if m != n:
            return False
        
        lps = compute_lps(pattern)
        i = 0  # index for line
        j = 0  # index for pattern

        while i < n:
            if pattern[j] == line[i]:
                i += 1
                j += 1

            if j == m:
                return True
            elif i < n and pattern[j] != line[i]:
                if j != 0:
                    j = lps[j - 1]
                else:
                    i += 1

        return False

    lines = text.split('\n')
    for line in lines:
        if kmp_search_line(pattern, line):
            return True

    return False

# Example usage
print('kmp_search (q1):', kmp_search(query_1, contents))  # Output: True
print('kmp_search (q2):', kmp_search(query_2, contents))  # Output: False

kmp_search (q1): True
kmp_search (q2): False


In [21]:
class AhoCorasick:
    def __init__(self):
        self.goto = {}
        self.output = {}
        self.fail = {}
        self.new_state = 0

    def add_pattern(self, pattern):
        state = 0
        for char in pattern:
            if (state, char) not in self.goto:
                self.new_state += 1
                self.goto[(state, char)] = self.new_state
            state = self.goto[(state, char)]
        self.output[state] = pattern

    def build_automaton(self):
        from collections import deque
        queue = deque()

        for char in {key[1] for key in self.goto if key[0] == 0}:
            state = self.goto[(0, char)]
            self.fail[state] = 0
            queue.append(state)

        while queue:
            r = queue.popleft()
            for key in {key[1] for key in self.goto if key[0] == r}:
                s = self.goto[(r, key)]
                queue.append(s)
                state = self.fail[r]
                while (state, key) not in self.goto and state != 0:
                    state = self.fail[state]
                if (state, key) in self.goto:
                    self.fail[s] = self.goto[(state, key)]
                else:
                    self.fail[s] = 0
                if self.fail[s] in self.output:
                    self.output[s] = self.output[self.fail[s]]

    def search(self, text):
        state = 0
        results = []
        for index, char in enumerate(text):
            while (state, char) not in self.goto and state != 0:
                state = self.fail[state]
            if (state, char) in self.goto:
                state = self.goto[(state, char)]
                if state in self.output:
                    results.append((index - len(self.output[state]) + 1, self.output[state]))
            else:
                state = 0
        return results

def aho_corasick_search(pattern, text):
    """
    Aho-Corasick algorithm to find a full line match of a pattern in the text.

    :param pattern: The pattern string to search for.
    :param text: The text in which to search for the pattern.
    :return: True if the pattern matches any full line in the text, otherwise False.
    """
    aho = AhoCorasick()
    aho.add_pattern(pattern)
    aho.build_automaton()

    lines = text.split('\n')
    for line in lines:
        if len(line) == len(pattern):
            matches = aho.search(line)
            for _, match in matches:
                if match == pattern:
                    return True

    return False

# Example usage
print('aho_corasick_search (q1):', aho_corasick_search(query_1, contents))  # Output: True
print('aho_corasick_search (q2):', aho_corasick_search(query_2, contents))  # Output: False

aho_corasick_search (q1): True
aho_corasick_search (q2): False


In [1]:
from fsearch.utils import benchmark_algorithms

file_path = "samples/200k.txt"
report_path = 'reports/benchmark.pdf'

benchmark_algorithms(file_path, report_path)



Algorithm            Time (seconds) 
-----------------------------------
Regex Search         0.018028       
Native Search        0.187474       
Rabin-Karp Search    0.282420       
Aho-Corasick Search  0.328304       
KMP Search           0.403537       
----------------------------------- 


Benchmark report saved to reports/benchmark.pdf
